In [3]:
#import spotipy
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd

In [4]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="dad67ad76b404a1cb4ff094d0146a762",
                                               client_secret="3f922e4f5a38455ab1304ec3a2c7e6a3",
                                               redirect_uri="http://localhost:3000",
                                               scope="user-library-read"))


In [5]:
# Warm & Fuzzy Playlist from Spotify
playlist_data = sp.user_playlist_tracks("spotify", "37i9dQZF1DX5IDTimEWoTd")
#print(playlist_data.keys())
num_tracks = len(playlist_data['items'])
tracks = []

#collect tracks ID 
for i in range (num_tracks):
    #toxic_id = toxic_id.replace("spotify:track:", "")
    song_id = playlist_data['items'][i]['track']['uri']
    song_id = song_id.replace("spotify:track:", "")
    tracks.append(song_id)

print(tracks)

['2769RhOZs0ezRncRW1vryp', '1Y3LN4zO1Edc2EluIoSPJN', '2IOFZdYYkFxEHVz1w34PoL', '1XNAH3h63O1hWFcgyHGbjf', '7Lv78wiIJcKgND9Zi7AIOt', '4A2LfnduSTsE8u0ecYROxE', '6I0KEvFfWszVJiCy5hum4V', '1aEsTgCsv8nOjEgyEoRCpS', '10UTmRvoroe6eZ3ViwkpbN', '79qxwHypONUt3AFq0WPpT9', '2PR4jVXzyjCky2PkONvipq', '6VsvKPJ4xjVNKpI8VVZ3SV', '0hiHNxqKGf3GZgVTIOAiiK', '4ZHGRFGwjnEIXHG2FhcDPv', '0OPIBKjAYBFzNpo1HzjqzQ', '49FwhOuCZ9LWaEpGNal2lQ', '1Mk9ou2m6hsEZJYuNRJFpO', '09fAL7YwPV3YzVmQDzLY8d', '0wavGRldH0AWyu2zvTz8zb', '6vC7mF2CXYW6MVF2voO4FU', '2XkDm5m2vPowecEAAR5gmb', '5odlY52u43F5BjByhxg7wg', '1hCTxutVVYvggAQcUUUZAT', '4XxUka4jIa0girDclOY4iA', '07koEqsKHZTlGVMC9eoEjO', '093KMwH96ZwZRZn2lKoX9Z', '1dGr1c8CrMLDpV6mPbImSI', '5iMUQTIxTGgbVItGIPAWbL', '3JjnGLK8IxkNLvo8Lb3KOM', '6FtIK0IkmG33lKbc1pyAlZ', '7b9oZ5tuGDKRWsL0lgQXqg', '6IPwKM3fUUzlElbvKw2sKl', '0YMe6PHRbeDcN7KJdCG0bW', '6qsacYwaDcrOy4CnTDxWNF', '0lx2cLdOt3piJbcaXIV74f', '4sFhD0PjpNgV8M8yIzt7C2', '1is8gU4RVcN4J8xItxWoOY', '727Z2S2mtNH46CbP9EWPwY', '2z3HKlpENv

In [6]:
def analyze_playlist(creator, playlist_id):
    
    # Create empty dataframe
    playlist_features_list = ["artist","album","track_name",  "track_id","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # Loop through every track in the playlist, extract features and append the features to the playlist df
    
    playlist = sp.user_playlist_tracks(creator, playlist_id)["items"]
    for track in playlist:
        # Create empty dict
        playlist_features = {}
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
        
    return playlist_df

In [7]:
analyzed_playlist_data = analyze_playlist("spotify", "37i9dQZF1DX5IDTimEWoTd")

/Users/victoria/Documents/me/MusicDataAnalysis/.env/lib/python3.5/site-packages/ipykernel_launcher.py:27: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [8]:
analyzed_playlist_data['danceability'].mean()

0.52921

In [9]:
# Compute the average values of each feature
averages = analyzed_playlist_data.mean()

# Calculate the variance of each feature
variances = analyzed_playlist_data.var()

print(variances)

danceability       0.018122
energy             0.024067
liveness           0.005745
loudness          10.447606
speechiness        0.000687
tempo           1201.278744
valence            0.020365
dtype: float64
